# Simple Example of a Pipe

Here is a simple example which constructs a pipe with radius $70.0$, thicknes $3.0$ and target element size of $30.0$.

It has a straight bit, bends in two axis with different radius, and then a straight bit.

In [5]:
import numpy as np
import pipefactory as pf

Straight0 = {
    'Name': 'Straight0',
    'length': 350.0,
    'type': 'Straight',
}

Bend1 = {
    'Name': 'Bend1',
    'type': 'Bend',
    'param': {
        'axis' : "left_right",
        'radius': 300.0,
        'angle': 95.0
    }
}

Bend2 = {
    'Name': 'Bend1',
    'type': 'Bend',
    'param': {
        'axis' : "up_down",
        'radius': 150.0,
        'angle': 45.0
    }
}

section_list = [Straight0, Bend1, Straight0, Bend2, Straight0]

mesh = pf.Pipe(radius = 70., 
               thickness = 3.0, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 6.0, 
               elements_through_thickness = 3)

mesh.export('foo.vtk')


In [ ]:
Bend1 = {
    'Name': 'Bend1',
    'type': 'Bend',
    'param': {
        'axis' : "left_right",
        'radius': 300.0,
        'angle': 95.0
    }
}

Bend2 = {
    'Name': 'Bend1',
    'type': 'Bend',
    'param': {
        'axis' : "up_down",
        'radius': 150.0,
        'angle': 45.0
    }
}

In [6]:
s0 = 150.0
phi0 = 45.0
A = 10.0 
ell = 25.0

mesh.add_defect_displacement(pf.Dimple(s0, phi0, A, ell))

point_data = {
    "u" : np.random.uniform(0.0, 1.0, size=(mesh.nnodes,))
}
mesh.export('foo_point.vtk', point_data=point_data)

### Example with a Defect

Here we show an example where we introduce a defect, in this case a dimple. In general any displacement defect or feature can be defined this way. 

A class of parameteric defects could be contructed. As suggested this could also be from a point cloud, where kNN could be used to interpolate points to return imperfection from perfect cylinder.

In [6]:
phi0 = 45.0
A = 10.0 
ell = 25.0

mesh.add_defect_displacement(pf.Dimple(150.0, phi0, A, ell))
mesh.add_defect_displacement(pf.Dimple(500.0, phi0, A, ell))
mesh.export("foo_multiple_defects.vtk")

In [5]:
import pipefactory as pf
import numpy as np

s0 = 150.0
phi0 = 45.0
A = 10.0 
ell = 25.0

Straight1 = {
    'Name': 'Straight0',
    'length': 350.0,
    'type': 'Straight',
}

mesh_with_defect = pf.Pipe(70.0, 3.0, [Straight1], ("hex", False), 10, 3)

mesh_with_defect.add_defect_displacement(pf.Dimple(s0, phi0, A, ell))

mesh_with_defect.export('foo_defect.vtk')

## How might it work with a point cloud?

Here we use a simple point based regession method (KNN) to interpolate from a point cloud.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import numpy as np


class PointCloud():

    def __init__(self,
                 filename : str,
                 nearest_neighbours : int = 3,
                 ):
        
        self.data = np.genfromtxt(filename, delimiter=',')

        self.X = self.data[:,:4] 
        self.y = self.data[:,:-1]

        self.knn = KNeighborsRegressor(n_neighbors=nearest_neighbours).fit(self.X, self.y)

    def __call__(self,
                 xm : np.array,
                 s : float, 
                 node : Node):
        
        new_data_point = np.array([s, node.v[0], node.v[1], node[2]])
        return self.knn.predict(new_data_point)

    

## Defect Example with Mesh Deletion

In [8]:
import pipefactory as pf

s0 = 150.0
phi0 = 45.0
radius = 25.0

Straight1 = {
    'Name': 'Straight0',
    'length': 350.0,
    'type': 'Straight',
}

mesh_with_defect = pf.Pipe(70.0, 3.0, [Straight1], ("hex", False), 2.0, 3)

#mesh_with_defect.remove_elements(pf.Hole(s0, phi0, radius))

mesh_with_defect.remove_elements(pf.Radial_Slit(s0=100.0, phi0=10., phi1=45.0, slit_width=3.0))

mesh_with_defect.export('foo_hole.vtk')



## Partition in the pipe into "Super Elements" for Generalised Multiscale Finite Elements

This code partitions the part into `nparts` it then overlaps through parts by a layer of `size_overlap`.

In [1]:
import numpy as np
import pipefactory as pf

s0 = 150.0
phi0 = 45.0
radius = 25.0

Straight1 = {
    'Name': 'Straight0',
    'length': 350.0,
    'type': 'Straight',
}

mesh_partition_test = pf.Pipe(radius = 70.0, 
                              thickness = 3.0, 
                              section_list = [Straight1], 
                              elem_type = ("hex", False), 
                              element_size = 2.0,
                              elements_through_thickness = 3, 
                              partition = True, 
                              nparts = 4, 
                              size_overlap = 2)


# Visualise the partitions as cell data.
cell_data = {}
for i, Omg in enumerate(mesh_partition_test.Omg):
    e = np.zeros((mesh_partition_test.nel,))
    e[Omg] = 1.0
    cell_data["Omg_"+str(i)] = [e] # This is a quirk of meshio plotting, this has to be a list of a list.

mesh_partition_test.export("foo_Omg.vtk", cell_data = cell_data)


In [91]:
import pipefactory as pf

le = pf.LinearElasticity()

Straight0 = {
    'Name': 'Straight0',
    'length': 350.0,
    'type': 'Straight',
}

mesh = pf.Pipe(radius = 70., thickness= 3.0, section_list=[Straight0], elem_type=("hex", False), element_size=10.0, elements_through_thickness=3)


In [4]:
import pipefactory as pf

mesh = pf.TwoBricks()

fe2 = pf.FEM(mesh)

lam, v = fe2.eig_decomposition(20)

# Visualise the partitions as cell data.
point_data = {}

# Iterate over each column
for i, v_col in enumerate(v.T):  # Transpose to iterate over columns
    point_data["eig_vec_" + str(i)] = v_col.reshape(12,3)

mesh.export("twobricks.vtk", point_data=point_data)



Print
Done1
Not here!


In [1]:
import random
import numpy as np
import pipefactory as pf

from scipy.sparse.linalg import eigsh


le = pf.LinearElasticity()

Straight0 = {
    'Name': 'Straight0',
    'length': 60.0,
    'type': 'Straight',
}

mesh = pf.Pipe(radius = 70., thickness= 3.0, section_list=[Straight0], elem_type=("hex", False), element_size=10.0, elements_through_thickness=3)

fe = pf.FEM(mesh)

#lam, v = fe.eig_decomposition(20)

K = fe.K_global.todense()

e, v = np.linalg.eigh(K)

# Visualise the partitions as cell data.
point_data = {}

# Iterate over each column
for i, v_col in enumerate(v.T):  # Transpose to iterate over columns
    if i == 20:
        break
    else: point_data["eig_vec_" + str(i)] = v_col.reshape(mesh.nnodes,3)
mesh.export("pipe_eigs.vtk", point_data=point_data)


In [12]:
# Visualise the partitions as cell data.
point_data = {}

# Iterate over each column
for i, v_col in enumerate(v.T):  # Transpose to iterate over columns
    point_data["eig_vec_" + str(i)] = v_col.reshape(mesh.nnodes,3)

mesh.export("pipe_eigs.vtk", point_data=point_data)

In [1]:
import pipefactory as pf

import numpy as np

Straight0 = {
    'Name': 'Straight0',
    'length': 60.0,
    'type': 'Straight',
}

radius = 70.
thickness = 3.0 

mesh = pf.Pipe(radius = radius, thickness= thickness, section_list=[Straight0], elem_type=("hex", False), element_size=10.0, elements_through_thickness=3)


mesh.export("foo.vtk")


In [5]:
mesh.outer_face_elements[3].midpoint

array([ 5.        , 34.17899594, 62.59415763])